# PT XYZ Data Warehouse Analysis
## Mining Operations Data Analysis

This notebook provides analysis of PT XYZ mining operations data including:
- Equipment usage patterns
- Production metrics
- Financial performance
- Operational efficiency

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyodbc
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")

In [ ]:
# Load data from CSV files for analysis
equipment_df = pd.read_csv('/home/jovyan/work/data/dataset_alat_berat_dw.csv')
production_df = pd.read_csv('/home/jovyan/work/data/dataset_production.csv')
transaction_df = pd.read_csv('/home/jovyan/work/data/dataset_transaksi.csv')

print(f"Equipment data: {len(equipment_df)} records")
print(f"Production data: {len(production_df)} records")
print(f"Transaction data: {len(transaction_df)} records")

In [ ]:
# Equipment Usage Analysis
print("=== EQUIPMENT USAGE ANALYSIS ===")
print("\nEquipment Types:")
print(equipment_df['equipment_type'].value_counts())

print("\nSites by Region:")
print(equipment_df.groupby('region')['site_name'].nunique())

# Equipment efficiency
equipment_df['efficiency'] = equipment_df['operating_hours'] / (equipment_df['operating_hours'] + equipment_df['downtime_hours']) * 100
print(f"\nAverage Equipment Efficiency: {equipment_df['efficiency'].mean():.2f}%")

In [ ]:
# Production Analysis
print("=== PRODUCTION ANALYSIS ===")
print("\nMaterial Types:")
print(production_df['material_type'].value_counts())

print("\nProduction by Region:")
region_production = production_df.groupby('region')['produced_volume'].sum().sort_values(ascending=False)
print(region_production)

print("\nEmployee Status Distribution:")
print(production_df['status'].value_counts())

In [ ]:
# Financial Analysis
print("=== FINANCIAL ANALYSIS ===")
print("\nBudget vs Actual Cost Analysis:")
transaction_df['cost_variance_pct'] = ((transaction_df['actual_cost'] - transaction_df['budgeted_cost']) / transaction_df['budgeted_cost']) * 100

print(f"Total Budgeted Cost: ${transaction_df['budgeted_cost'].sum():,.2f}")
print(f"Total Actual Cost: ${transaction_df['actual_cost'].sum():,.2f}")
print(f"Total Variance: ${transaction_df['variance'].sum():,.2f}")
print(f"Average Cost Variance: {transaction_df['cost_variance_pct'].mean():.2f}%")

In [ ]:
# Visualizations
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Equipment efficiency by type
equipment_eff = equipment_df.groupby('equipment_type')['efficiency'].mean().sort_values(ascending=False)
axes[0,0].bar(equipment_eff.index, equipment_eff.values)
axes[0,0].set_title('Average Equipment Efficiency by Type')
axes[0,0].set_ylabel('Efficiency (%)')
axes[0,0].tick_params(axis='x', rotation=45)

# Production by material type
material_prod = production_df.groupby('material_type')['produced_volume'].sum()
axes[0,1].pie(material_prod.values, labels=material_prod.index, autopct='%1.1f%%')
axes[0,1].set_title('Production Volume by Material Type')

# Cost variance by project
project_variance = transaction_df.groupby('project_name')['variance'].sum().sort_values()
axes[1,0].barh(project_variance.index, project_variance.values)
axes[1,0].set_title('Cost Variance by Project')
axes[1,0].set_xlabel('Variance ($)')

# Monthly production trend (if date data is available)
production_df['date'] = pd.to_datetime(production_df['date'])
monthly_prod = production_df.groupby(production_df['date'].dt.to_period('M'))['produced_volume'].sum()
axes[1,1].plot(monthly_prod.index.astype(str), monthly_prod.values, marker='o')
axes[1,1].set_title('Monthly Production Trend')
axes[1,1].set_ylabel('Produced Volume')
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Database Connection Test (when available)
def connect_to_datawarehouse():
    """
    Connect to the SQL Server data warehouse
    """
    try:
        connection_string = (
            "DRIVER={ODBC Driver 17 for SQL Server};"
            "SERVER=sqlserver,1433;"
            "DATABASE=PTXYZ_DataWarehouse;"
            "UID=sa;"
            "PWD=PTXYZDataWarehouse2025;"
            "TrustServerCertificate=yes;"
        )
        conn = pyodbc.connect(connection_string)
        return conn
    except Exception as e:
        print(f"Connection failed: {e}")
        return None

# Test connection
print("Testing database connection...")
conn = connect_to_datawarehouse()
if conn:
    print("✅ Database connection successful!")
    
    # Query some basic stats from the data warehouse
    query = """
    SELECT 
        'Equipment Usage' as table_name, COUNT(*) as record_count
    FROM staging.EquipmentUsage
    UNION ALL
    SELECT 
        'Production' as table_name, COUNT(*) as record_count
    FROM staging.Production
    UNION ALL
    SELECT 
        'Financial Transactions' as table_name, COUNT(*) as record_count
    FROM staging.FinancialTransaction
    """
    
    try:
        df_stats = pd.read_sql(query, conn)
        print("\nData Warehouse Table Counts:")
        print(df_stats)
    except Exception as e:
        print(f"Query failed: {e}")
    
    conn.close()
else:
    print("❌ Database connection failed - using CSV data only")

## Key Insights

Based on the analysis above, here are the key insights from PT XYZ mining operations:

1. **Equipment Efficiency**: Monitor equipment types with lower efficiency rates
2. **Production Patterns**: Identify peak production periods and material demand
3. **Cost Management**: Track projects with significant cost variances
4. **Regional Performance**: Compare performance across different mining regions

## Next Steps

1. Set up automated ETL pipeline in Airflow
2. Create real-time dashboards in Grafana/Superset
3. Implement alerting for equipment downtime
4. Develop predictive maintenance models